In [1]:
from urllib.request import urlopen
from collections import OrderedDict
url = "https://sheets.googleapis.com/$discovery/rest?version=v4"

r = urlopen(url).read()
import json
j=json.loads(r.decode(), object_hook=OrderedDict)

def dump_l(l, indent=0):
    for item in l:
        if isinstance(item, list):
            dump_l(item, indent+1)
        elif isinstance(item, dict):
            dump_d(item, indent+1)
        else:
            print(indent*" ", item, sep="")
        
def dump_d(d, indent=0):
    for k, v in d.items():
        if isinstance(v, dict):
            print(indent*" ", k, ":", sep="")
            dump_d(v, indent+1)
        elif isinstance(v, list):
            dump_l(v, indent+1)
        else:
            assert isinstance(v, (int, str)), type(v)
            print(indent*" ", k, ': ', v, sep="")
            
def dump_resources(d, indent=0):
    for k, v in d.items():
        if k == 'resources':
            for k2, v2 in v.items():
                print(indent*" ", k2, ":", sep="")
                dump_resources(v2, indent+1)

In [2]:
dump_resources(j)

spreadsheets:
 sheets:
 values:


In [3]:
%cd ../pyprojects/goffice3

C:\Users\Administrator\Documents\pyprojects\goffice3


In [12]:
# %load sheets/api.py
"""

Created by: Nathan Starkweather
Created on: 06/10/2016
Created in: PyCharm Community Edition


"""
__author__ = 'Nathan Starkweather'

import logging

logger = logging.getLogger(__name__)
_h = logging.StreamHandler()
_f = logging.Formatter("%(created)s %(name)s %(levelname)s (%(lineno)s): %(message)s")
_h.setFormatter(_f)
logger.addHandler(_h)
logger.propagate = False
logger.setLevel(logging.DEBUG)
del _h, _f

from goffice3 import GSession
from goffice3 import GObject


SPREADSHEETS_URL_BASE = "https://sheets.googleapis.com/v4/spreadsheets/"

def ujoin(*args): return "/".join(args)
def qjoin(*args): return "&".join(args)
def eqjoin(*args): return "=".join(args)


def make_query(options):
    kvs = [eqjoin(k, v) for k, v in options if v is not None]
    if not kvs:
        return ""
    return qjoin(*kvs)


class Dimension(GObject):
    """ Dimension enum """
    DIMENSION_UNSPECIFIED = "DIMENSION_UNSPECIFIED"
    ROWS = "ROWS"
    COLUMNS = "COLUMNS"


class ValueRenderOption(GObject):
    """ ValueRenderOption enum """
    FORMATTED_VALUE = "FORMATTED_VALUE"
    UNFORMATTED_VALUE = "UNFORMATTED_VALUE"
    FORMULA = "FORMULA"


class DateTimeRenderOption(GObject):
    SERIAL_NUMBER = "SERIAL_NUMBER"
    FORMATTED_STRING = "FORMATTED_STRING"


# brevity
D = Dimension
V = ValueRenderOption
DT = DateTimeRenderOption


class GSheetsAPI(GObject):
    """ This API layer implements the raw google sheets
    v4 api via 1-1 translation from python to the REST
    api endpoint(s).

    This api is defined as a hierarchy of collections in
    the same way as the actual google api, to make
    implementation directly reflect the google api structure.

    All Api subobjects (spreadsheets, values...) retain reference
    to the top level api object as a just-in-case implemented in early
    programming.
    """

    def __init__(self, session):
        self.session = session
        self.spreadsheets = SpreadsheetsAPICollection(session, self)

    @classmethod
    def from_creds(cls, creds):
        return cls(GSession(creds))


class SpreadsheetsAPICollection(GObject):
    def __init__(self, session, api):
        self.session = session
        self.api = api
        self.sheets = SheetsAPICollection(session, api)
        self.values = ValuesAPICollection(session, api)

    def batchUpdate(self, ssid, updates):
        raise NotImplementedError

    def create(self):
        raise NotImplementedError

    def get(self):
        raise NotImplementedError


class ValuesAPICollection(GObject):
    def __init__(self, session, api):
        self.session = session
        self.api = api

    def batchGet(self, ssid):
        raise NotImplementedError

    def batchUpdate(self, ssid):
        raise NotImplementedError

    def get(self, ssid, ss_range, majorDimension=None, valueRenderOption=None,
            dateTimeRenderOption=None):
        query_options = (
            ('majorDimension', majorDimension),
            ('valueRenderOption', valueRenderOption),
            ('dateTimeRenderOption', dateTimeRenderOption)
        )
        query = make_query(query_options)

        url = ujoin(SPREADSHEETS_URL_BASE, str(ssid), "values", ss_range, query)
        return self.session.get(url)

    def update(self, ssid, ss_range):
        raise NotImplementedError


class SheetsAPICollection(GObject):
    def __init__(self, session, api):
        self.session = session
        self.api = api

    def copyTo(self):
        raise NotImplementedError


In [7]:
import os
from goffice3 import load_creds
import json
os.chdir(os.path.expanduser("~/Documents/pyprojects/google_auths/.auth"))
s = GSession(load_creds('_exp_auth_key.json'))
api = GSheetsAPI(s)

dv3u = "https://www.googleapis.com/drive/v3"
dv3fileslist = dv3u+"/files"
r, c = s.get(dv3fileslist)
j=json.loads(c.decode())

ssid = j['files'][0]['id']
j['files']

1465618041.266726 goffice3.baseobject DEBUG (47): New instance of <GSession>.
1465618041.270726 goffice3.baseobject DEBUG (47): New instance of <GSheetsAPI>.
1465618041.271726 goffice3.baseobject DEBUG (47): New instance of <SpreadsheetsAPICollection>.
1465618041.273726 goffice3.baseobject DEBUG (47): New instance of <SheetsAPICollection>.
1465618041.275727 goffice3.baseobject DEBUG (47): New instance of <ValuesAPICollection>.
1465618041.451738 goffice3.session DEBUG (62): GET: https://www.googleapis.com/drive/v3/files


[{'id': '10x9ziXbfVm5ZcchAZjW1Zi7L0c0tDJfPhIvsnryvQZY',
  'kind': 'drive#file',
  'mimeType': 'application/vnd.google-apps.spreadsheet',
  'name': 'Slayer Tasks'},
 {'id': '0B85fKUTk3Dmbc3RhcnRlcl9maWxl',
  'kind': 'drive#file',
  'mimeType': 'application/pdf',
  'name': 'How to get started with Drive'}]

In [13]:
try:
    api.spreadsheets.values.get(ssid, "A5")
except 

1465618310.502929 goffice3.session DEBUG (62): GET: https://sheets.googleapis.com/v4/spreadsheets//10x9ziXbfVm5ZcchAZjW1Zi7L0c0tDJfPhIvsnryvQZY/values/A5/


GSessionError: 404: b'<!DOCTYPE html>\n<html lang=en>\n  <meta charset=utf-8>\n  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">\n  <title>Error 404 (Not Found)!!1</title>\n  <style>\n    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) no-repeat 0% 0%/100% 100%;